# Loading data from Digital Earth Africa <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">


In [ ]:
# pip install dist/deafrica_tools-0.1.0.tar.gz

In [1]:
import datacube
from odc.ui import with_ui_cbk
import datacube
import matplotlib.pyplot as plt
import datacube.utils.rio
from datacube.utils import masking
import odc.algo

from deafrica_tools.plotting import rgb
from deafrica_tools.datahandling import load_ard
from deafrica_tools.dask import create_local_dask_cluster

#This will speed up loading data
datacube.utils.rio.set_default_rio_config(aws='auto', cloud_defaults=True)

%load_ext autoreload
%autoreload 2

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:37045 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 2 Memory: 13.11 GB


### Connect to the datacube
We then need to connect to the datacube database.
We will then be able to use the `dc` datacube object to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [3]:
dc = datacube.Datacube(app="03_Loading_data")

## query

In [7]:
lat, lon = 33.4291, -7.6058
buffer = 0.1
measurements=['red','green','blue', 'swir_1']#, 'surface_temperature', 'atmospheric_transmittance']

# Create a reusable query
query = {
    'x': (lon-buffer, lon+buffer),
    'y': (lat+buffer, lat-buffer),
    'time': ('2018-01'),
    'resolution': (-30, 30),
    'dask_chunks':{},
    'output_crs':'epsg:6933',
}

## load_ard

In [9]:
ld = load_ard(dc=dc,
         products=['s2_l2a'],# ['ls5_c2l2','ls7_c2l2','ls8_c2l2'],
#          measurements=measurements,
         min_gooddata=0.1,
         verbose=True,
         **query)

print(ld)

/env/lib/python3.6/site-packages/deafrica_tools/datahandling.py:244: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  "Setting 'min_gooddata' percentage to > 0.0 "


Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 1 out of 1 time steps with at least 10.0% good quality pixels
Applying pixel quality/cloud mask
Returning 1 time steps as a dask array
<xarray.Dataset>
Dimensions:      (time: 1, x: 644, y: 714)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-02T11:19:01
  * y            (y) float64 4.043e+06 4.043e+06 ... 4.022e+06 4.022e+06
  * x            (x) float64 -7.435e+05 -7.435e+05 ... -7.242e+05 -7.242e+05
    spatial_ref  int32 6933
Data variables:
    B01          (time, y, x) float32 dask.array<chunksize=(1, 714, 644), meta=np.ndarray>
    B02          (time, y, x) float32 dask.array<chunksize=(1, 714, 644), meta=np.ndarray>
    B03          (time, y, x) float32 dask.array<chunksize=(1, 714, 644), meta=np.ndarray>
    B04          (time, y, x) float32 dask.array<chunksize=(1, 714, 644), meta=np.ndarray>
    B05          (time, y, x) float32 da

In [10]:
ld.compute()

<xarray.Dataset>
Dimensions:      (time: 1, x: 644, y: 714)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-02T11:19:01
  * y            (y) float64 4.043e+06 4.043e+06 ... 4.022e+06 4.022e+06
  * x            (x) float64 -7.435e+05 -7.435e+05 ... -7.242e+05 -7.242e+05
    spatial_ref  int32 6933
Data variables:
    B01          (time, y, x) float32 nan nan nan nan ... 135.0 135.0 135.0
    B02          (time, y, x) float32 nan nan nan nan ... 349.0 595.0 397.0
    B03          (time, y, x) float32 nan nan nan nan ... 572.0 935.0 703.0
    B04          (time, y, x) float32 nan nan nan nan ... 845.0 1162.0 835.0
    B05          (time, y, x) float32 nan nan nan nan ... 1081.0 1232.0 1710.0
    B06          (time, y, x) float32 nan nan nan nan ... 1179.0 1518.0 2151.0
    B07          (time, y, x) float32 nan nan nan nan ... 1238.0 1644.0 2372.0
    B08          (time, y, x) float32 nan nan nan nan ... 1490.0 2600.0 2254.0
    B8A          (time, y, x) float32 nan nan nan nan ... 1461.0 1822.0 2519.0
    B09          (time, y, x) float32 nan nan nan nan ... 1953.0 1953.0 1953.0
    B11          (time, y, x) float32 nan nan nan nan ... 2524.0 2641.0 2790.0
    B12          (time, y, x) float32 nan nan nan nan ... 2274.0 2277.0 2238.0
    AOT          (time, y, x) float32 nan nan nan nan ... 197.0 197.0 197.0
    WVP          (time, y, x) float32 nan nan nan nan ... 509.0 718.0 718.0
    SCL          (time, y, x) uint8 0 0 0 0 0 0 0 0 0 0 ... 5 4 4 4 4 5 5 5 5 5
Attributes:
    crs:           epsg:6933
    grid_mapping:  spatial_ref

In [ ]:
rgb(ld, index=[1,3,5,10,-1])

In [ ]:
ld.swir_1.plot(col='time', col_wrap=5)

## dc.load

In [ ]:
ds = dc.load(product='ls8_c2l2',
#              measurements=measurements,
             **query
            ).compute()

# ds = dc.load(product='ls7_c2l2',
#              measurements=measurements,
#              **query
#             )

# ds = dc.load(product='ls5_c2l2',
# #              measurements=measurements,
#              **query
#             )


In [ ]:
# for band in ds.data_vars:
#     fig,ax=plt.subplots(1,1, figsize=(5,5))
#     ds[band].isel(time=2).plot()
#     ax.set_title(band)

In [ ]:
# ds.QA_PIXEL.flags_definition

In [ ]:
import pprint
pprint.pprint(ds.QA_PIXEL.flags_definition)

In [ ]:
quality_flags_prod = {
                "clear": True,
                "cloud_shadow": "not_high_confidence",
                "nodata": False,
            }

# "cirrus":"not_high_confidence"
pq_mask = masking.make_mask(ds['QA_PIXEL'], **quality_flags_prod)


In [ ]:
ds_masked = odc.algo.keep_good_only(ds, where=pq_mask)#.compute()
ds_masked = odc.algo.to_float(ds_masked, dtype='float32').compute()

In [ ]:
st = ds_masked['ST_B6'] * 0.00341802 + 149.0 - 273.15

In [ ]:
for i in range(len(ds.time)):
    fig,ax=plt.subplots(1,4, figsize=(15,5), sharey=True)
    pq_mask.isel(time=i).plot(ax=ax[1], add_colorbar=False)
    rgb(ds, index=i, ax=ax[0])
    rgb(ds_masked, index=i, ax=ax[2])
    st.isel(time=i).plot(ax=ax[3])   
       